---
title: "Market Research"
execute: 
  freeze: false
  python: .venv/bin/python
---




This page contains an outline of the topics, content, and assignments for the semester. Note that this schedule will be updated as the semester progresses and the timeline of topics and assignments might be updated throughout the semester.


In [ ]:
#| echo: false
#| warning: false
import pandas as pd
from great_tables import GT
import requests
from dotenv import load_dotenv
import os
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Get API key and spreadsheet ID from environment variables
API_KEY = os.getenv("GOOGLE_SHEETS_API_KEY")
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")
SHEET_NAME = os.getenv("SHEET_NAME", "")  # Try to get sheet name from env

def get_sheet_names():
    """Get a list of sheet names in the spreadsheet"""
    if not API_KEY or not SPREADSHEET_ID:
        return []
    
    url = f"https://sheets.googleapis.com/v4/spreadsheets/{SPREADSHEET_ID}?key={API_KEY}"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return []
        
        data = response.json()
        sheets = data.get('sheets', [])
        return [sheet['properties']['title'] for sheet in sheets]
    except Exception:
        return []

# Determine the sheet name to use
if not SHEET_NAME:
    sheet_names = get_sheet_names()
    if sheet_names:
        SHEET_NAME = sheet_names[0]  # Use the first sheet if available
    else:
        SHEET_NAME = ""  # Will try without sheet name

# Construct the range with sheet name if available
if SHEET_NAME:
    RANGE_NAME = f"{SHEET_NAME}!A1:G100"
else:
    RANGE_NAME = "A1:G100"  # Try with default sheet

print(f"Using range: {RANGE_NAME}")

# Call the Sheets API
url = f"https://sheets.googleapis.com/v4/spreadsheets/{SPREADSHEET_ID}/values/{RANGE_NAME}?key={API_KEY}"
print(f"Connecting to Google Sheets API...")
response = requests.get(url)

# Add error handling to diagnose issues
if response.status_code != 200:
    print(f"Error: API request failed with status code {response.status_code}")
    
    # Special handling for permission error
    if response.status_code == 403 and "PERMISSION_DENIED" in response.text:
        print("\nPERMISSION ERROR: The API key doesn't have permission to access this spreadsheet.")
        print("To fix this issue, you need to make your spreadsheet publicly accessible:")
        print("1. Open your spreadsheet in Google Sheets")
        print("2. Click 'Share' button in the top right")
        print("3. Click 'Change to anyone with the link'")
        print("4. Make sure 'Viewer' is selected")
        print("5. Click 'Done'")
        
        # Create a placeholder table with error message
        df = pd.DataFrame({
            'week': [''],
            'date': [datetime.now()],
            'topic': ['Error loading schedule from Google Sheets. Please check the console output for details.'],
            'prepare': [''],
            'materials': [''],
            'due': [''],
        })
    elif response.status_code == 400 and "Unable to parse range" in response.text:
        print("\nSHEET NAME ERROR: The sheet name doesn't exist in the spreadsheet.")
        print("Check your spreadsheet and update the SHEET_NAME in your .env file.")
        print(f"Available sheets: {', '.join(get_sheet_names()) or 'Could not retrieve'}")
        
        # Create a placeholder table with error message
        df = pd.DataFrame({
            'week': [''],
            'date': [datetime.now()],
            'topic': ['Error: Invalid sheet name. Check the console output for details.'],
            'prepare': [''],
            'materials': [''],
            'due': [''],
        })
    else:
        print(f"Response error: {response.text}")
        # Create a placeholder table
        df = pd.DataFrame({
            'week': [''],
            'date': [datetime.now()],
            'topic': ['Error loading schedule. Check console for details.'],
            'prepare': [''],
            'materials': [''],
            'due': [''],
        })
else:
    result = response.json()
    # Debug information
    print(f"API connection successful")
    
    values = result.get('values', [])
    if not values:
        print('No data found in the spreadsheet.')
        # Create a placeholder table
        df = pd.DataFrame({
            'week': [''],
            'date': [datetime.now()],
            'topic': ['No data found in spreadsheet.'],
            'prepare': [''],
            'materials': [''],
            'due': [''],
        })
    else:
        print(f"Found {len(values)-1} data rows")
        headers = values[0]  # First row is headers
        data = values[1:]    # Rest is data
        
        # Check for inconsistent column counts
        expected_column_count = len(headers)
        print(f"Expected {expected_column_count} columns: {headers}")
        
        # Fix rows with missing columns by adding empty strings
        fixed_data = []
        for i, row in enumerate(data):
            if len(row) < expected_column_count:
                print(f"Row {i+1} has only {len(row)} columns, adding empty values")
                # Add empty strings for missing columns
                fixed_row = row + [''] * (expected_column_count - len(row))
                fixed_data.append(fixed_row)
            else:
                fixed_data.append(row)
        
        # Create DataFrame with the fixed data
        df = pd.DataFrame(fixed_data, columns=headers)
        
        # Clean up data similar to R code
        if 'dow' in df.columns:
            df = df.drop(columns=['dow'])
        if 'session' in df.columns:
            df = df.drop(columns=['session'])
        
        # Keep the original date strings for display
        if 'date' in df.columns:
            df['original_date'] = df['date']
            
            # Check if dates have year component
            sample_date = df['date'].iloc[0] if not df['date'].empty else ""
            has_year = any(str(year) in sample_date for year in range(2000, 2100))
            
            if not has_year:
                # If dates don't have year, add the current year for proper parsing
                current_year = datetime.now().year
                print(f"Dates in spreadsheet don't have year component. Adding {current_year} for parsing.")
                
                # For dates like "Jan. 11", convert to "Jan. 11, 2024" format
                df['date_with_year'] = df['date'].apply(
                    lambda x: f"{x}, {current_year}" if x and not any(str(y) in x for y in range(2000, 2100)) else x
                )
                
                try:
                    # Parse with year added - add dayfirst=True to silence warning
                    df['date'] = pd.to_datetime(df['date_with_year'], errors='coerce', dayfirst=True)
                    print("Successfully parsed dates with added year")
                except Exception as e:
                    print(f"Error parsing dates: {str(e)}")
                    # Keep original strings
                    df['date'] = df['original_date']
            else:
                try:
                    # Standard date parsing - add dayfirst=True to silence warning
                    df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)
                except Exception as e:
                    print(f"Error parsing dates: {str(e)}")
                    # Keep original strings if parsing fails
                    pass

# Replace NaN with empty strings for formatting
df = df.fillna("")

# Print the actual columns available in the dataframe to help debug
print(f"Available columns in dataframe: {df.columns.tolist()}")

# Create the GT table with similar styling to the R version
gt_table = GT(df)

# Format the dates - if using original strings, don't format as dates
if 'date' in df.columns and pd.api.types.is_datetime64_dtype(df['date']):
    # Successfully parsed as dates, use date formatting with a valid date_style
    # Use "day_m" which corresponds to format like "Thu, Jan 11"
    gt_table = gt_table.fmt_date(columns="date", date_style="day_m")
    print("Using date formatting with style 'day_m'")
else:
    # Using original strings, no date formatting needed
    print("Using original date strings")

# Continue with the rest of your table formatting
# Only use columns that actually exist in the dataframe
actual_columns = df.columns.tolist()

# Create column width specification using only columns that exist in the data
column_widths = {}
if 'week' in actual_columns:
    column_widths['week'] = "45px"
if 'date' in actual_columns:
    column_widths['date'] = "100px"
if 'prepare' in actual_columns:
    column_widths['prepare'] = "100px"
if 'topic' in actual_columns:
    column_widths['topic'] = "300px"
if 'materials' in actual_columns:
    column_widths['materials'] = "150px"
if 'due' in actual_columns:
    column_widths['due'] = "300px"

# Create lists of columns for alignment and markdown formatting
center_align_cols = [col for col in ['week'] if col in actual_columns]
right_align_cols = [col for col in ['date'] if col in actual_columns]
left_align_cols = [col for col in ['topic', 'prepare', 'materials', 'due'] if col in actual_columns]
markdown_cols = [col for col in ['topic', 'prepare', 'materials', 'due'] if col in actual_columns]

# Generate the table with dynamic column references
(
    gt_table
    .cols_align(
        align="center",
        columns=center_align_cols if center_align_cols else []
    )
    .cols_align(
        align="right",
        columns=right_align_cols if right_align_cols else []
    )
    .cols_align(
        align="left",
        columns=left_align_cols if left_align_cols else []
    )
    .fmt_markdown(
        columns=markdown_cols if markdown_cols else []
    )
    .cols_width(
        spec=column_widths
    )
    .cols_label(
        **{col: f"**{col.upper()}**" for col in actual_columns}
    )
    .tab_options(
        table_font_size=13
    )
    .opt_row_striping()
)